In [13]:
from datetime import datetime
import pandas as pd
import numpy as np

def merge_select_topI(doc, aux_data, content,onKey = 'document_id'):
    doc_aux = doc.merge(aux_data, how = 'left', on = onKey)
    doc_aux.fillna(-99, inplace= True)
    doc_aux[content+ "_topI"] = doc_aux[content + "_conf"].apply(getTopOne)
    return doc_aux

def getTopOne(l):
    if l == -99:
        return -99
    else:
        l = sorted(l, key = lambda x :x[1], reverse=True)
        return l[0][0]
    
def convert_ts(timestamp_ms_relative):
    TIMESTAMP_DELTA=1465876799998
    return datetime.fromtimestamp((int(timestamp_ms_relative)+TIMESTAMP_DELTA)//1000)

In [9]:
meta = pd.read_csv("../data/documents_meta.csv.gz", index_col=False)
cate = pd.read_csv("../data/documents_categories.csv.gz", index_col = False)
enti = pd.read_csv("../data/documents_entities.csv.gz", index_col= False)
topi = pd.read_csv("../data/documents_topics.csv.gz", index_col= False)

In [20]:
topi["topic_conf"] = topi[["topic_id", "confidence_level"]].apply(tuple, axis=1)
enti["entity_conf"] = enti[["entity_id", "confidence_level"]].apply(tuple, axis=1)
cate["categ_conf"] = cate[["category_id", "confidence_level"]].apply(tuple, axis=1)

topics = topi[["document_id", "topic_conf"]].groupby(["document_id"])["topic_conf"].apply(list).reset_index()
entities = enti[["document_id", "entity_conf"]].groupby(["document_id"])["entity_conf"].apply(list).reset_index()
categories = cate[["document_id", "categ_conf"]].groupby(["document_id"])["categ_conf"].apply(list).reset_index()

In [2]:
partition00 = pd.read_csv("../input/pages_view_uid0.csv.gz", index_col=False)
partition00["timestamp"] = partition00["timestamp"].map(convert_ts)
doc_ids = partition00[["document_id"]].drop_duplicates()

In [15]:
doc_meta = meta.merge(doc_ids, how = 'left', on = 'document_id')

In [33]:
doc_topics = doc_meta.merge(topics, how = 'left', on = 'document_id')
doc_topics.head()

,document_id,source_id,publisher_id,publish_time,topic_conf
0,1595802,1.0,603.0,2016-06-05 00:00:00,"[(140.0, 0.0731131601069), (16.0, 0.0594164867..."
1,1524246,1.0,603.0,2016-05-26 11:00:00,"[(113.0, 0.19645040221), (260.0, 0.14287816210..."
2,1617787,1.0,603.0,2016-05-27 00:00:00,"[(113.0, 0.216892316441), (260.0, 0.0963125069..."
3,1615583,1.0,603.0,2016-06-07 00:00:00,"[(89.0, 0.316306499636), (198.0, 0.01572182397..."
4,1615460,1.0,603.0,2016-06-20 00:00:00,"[(260.0, 0.0979641463762), (26.0, 0.0145508920..."


In [36]:
doc_topics.fillna(-99, inplace= True)

In [38]:
doc_topics["topic_topI"] = doc_topics["topic_conf"].apply(getTopOne)

In [39]:
doc_categ = merge_select_topI(doc_topics, categories, content = "categ")
doc_entity = merge_select_topI(doc_categ, entities, content = "entity")
doc_entity.drop(['topic_conf', 'categ_conf', 'entity_conf'], axis = 1, inplace = True)

In [47]:
partition00 = partition00.merge(doc_entity, how = 'left', on = 'document_id')

In [55]:
partition00.sort(['uuid']).head(200)

/home/ubuntu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,uuid,document_id,timestamp,platform,geo_location,traffic_source,source_id,publisher_id,publish_time,topic_topI,categ_topI,entity_topI
11707848,100001174c8854,1438654,2016-06-22 16:30:57,1,US>CA>803,1,5315.0,1046.0,2016-05-14 00:00:00,265.0,1408.0,3e2329d0532b88665cdd7c9625786738
1695938,100001174c8854,1856810,2016-06-15 17:43:38,1,US>CA>803,1,574.0,1161.0,2016-06-14 20:00:00,147.0,1902.0,543a19352803579575fe4a63dae66498
4576057,100001174c8854,2110889,2016-06-17 18:44:01,1,US>CA>803,1,694.0,1161.0,2016-06-17 14:00:00,183.0,1909.0,-99
11014727,1000018be5d703,2240510,2016-06-22 10:07:40,2,US>FL>528,1,663.0,61.0,2016-06-19 12:00:00,216.0,1702.0,9ebd449f947b6a9358f11aa77f3ce0d2
17836402,10000380c5ae26,2819961,2016-06-27 18:21:39,2,US>CA>803,1,1893.0,160.0,2016-06-28 00:00:00,184.0,1902.0,9e6e6bfa404b26cfb3012bd457fbdb68
6585486,10000380c5ae26,2185512,2016-06-19 00:37:10,2,US>CA>825,1,752.0,43.0,2016-06-18 10:00:00,51.0,1908.0,-99
4790241,10000380c5ae26,2112875,2016-06-17 18:27:58,2,US>CA>803,1,465.0,160.0,2016-06-17 08:00:00,77.0,1702.0,f53e449e38e3c2b4f1eadbd1e76a5715
13712587,10000489b154a3,2655482,2016-06-24 14:46:36,1,US>MI>563,1,7344.0,874.0,2016-06-23 00:00:00,45.0,1903.0,-99
10239173,100007fce4ea8f,394689,2016-06-21 17:44:56,1,US>MA>506,1,7343.0,874.0,-99,147.0,2006.0,06c0312fcb3a1445daeb4efd64952503
2584751,10000937259378,1477062,2016-06-15 11:50:57,2,US>NY>501,1,11339.0,1216.0,-99,174.0,1503.0,-99
